In [25]:
# Imports
import pandas as pd
import numpy as np
import networkx
box_dir = '/Users/plutzer/Box'

In [26]:
# Loading files
proteinGroups = pd.read_csv('/Users/plutzer/majlab/linkedFDR_testOutput.txt',delimiter = '\t')
proteinGroups

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (30,31,32,48,49,69,70,71,93,94,95,117,118,119,141,142,143,165,166,167,189,190,191) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,Protein IDs,Majority protein IDs,Peptide counts (all),Peptide counts (razor+unique),Peptide counts (unique),Fasta headers,Number of proteins,Peptides,Razor + unique peptides,...,Intensity mt_NLS_1,Intensity mt_NLS_2,Intensity mt_NLS_3,LFQ intensity mt_NLS_1,LFQ intensity mt_NLS_2,LFQ intensity mt_NLS_3,MS/MS count mt_NLS_1,MS/MS count mt_NLS_2,MS/MS count mt_NLS_3,FDR
0,0,CON__ENSEMBL:ENSBTAP00000024146,CON__ENSEMBL:ENSBTAP00000024146,3,3,3,NaN,1,3,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.069256
1,1,sp|O43790|KRT86_HUMAN;CON__O43790;sp|Q14533|KR...,sp|O43790|KRT86_HUMAN;CON__O43790;sp|Q14533|KR...,14;14;13;13,13;13;12;12,2;2;2;2,"sp|O43790|KRT86_HUMAN Keratin, type II cuticul...",4,14,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.057249
2,2,sp|O76009|KT33A_HUMAN;CON__Q6NTB9;CON__O76009,sp|O76009|KT33A_HUMAN;CON__Q6NTB9;CON__O76009,15;15;15,2;2;2,2;2;2,"sp|O76009|KT33A_HUMAN Keratin, type I cuticula...",3,15,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.064841
3,3,CON__P00761,CON__P00761,7,7,7,NaN,1,7,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.057249
4,4,CON__P02533;sp|P02533|K1C14_HUMAN;CON__Q6IFX2,CON__P02533;sp|P02533|K1C14_HUMAN,10;10;4,3;3;1,2;2;1,";sp|P02533|K1C14_HUMAN Keratin, type I cytoske...",3,10,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.057249
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24976,3362,sp|Q9Y6V7|DDX49_HUMAN,sp|Q9Y6V7|DDX49_HUMAN,2,2,2,sp|Q9Y6V7|DDX49_HUMAN Probable ATP-dependent R...,1,2,2,...,4755300.0,16033000.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,0.087173
24977,3363,sp|Q9Y6W5|WASF2_HUMAN,sp|Q9Y6W5|WASF2_HUMAN,4,4,4,sp|Q9Y6W5|WASF2_HUMAN Wiskott-Aldrich syndrome...,1,4,4,...,9515800.0,31612000.0,43033000.0,11809000.0,13509000.0,14499000.0,2.0,1.0,3.0,0.065704
24978,3364,sp|Q9Y6X4|F169A_HUMAN,sp|Q9Y6X4|F169A_HUMAN,1,1,1,sp|Q9Y6X4|F169A_HUMAN Soluble lamin-associated...,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.068139
24979,3365,sp|Q9Y6Y0|NS1BP_HUMAN,sp|Q9Y6Y0|NS1BP_HUMAN,1,1,1,sp|Q9Y6Y0|NS1BP_HUMAN Influenza virus NS1A-bin...,1,1,1,...,1226800.0,0.0,4264400.0,0.0,0.0,0.0,1.0,1.0,1.0,0.106980


In [31]:
# Get a list of all protein IDs in the groups
groupedIDs = list(proteinGroups['Protein IDs'])
proteinIDs = list(set([Ids for groups in groupedIDs for Ids in groups.split(';')]))
len(proteinIDs)

6628

In [32]:
# Make an adjacency matrix for the protein groups based on shared proteins:
# 0 = no shared proteins between groups
# 1 = shared proteins between groups
adjMat = np.zeros((len(groupedIDs),len(groupedIDs)))
for group_index in range(len(groupedIDs)):
    for protein in groupedIDs[group_index].split(';'):
        also_in  = list(np.where([protein in group.split(';') for group in groupedIDs])[0])
        print(also_in)
        for index in also_in:
            if group_index != index:
                #print('Match: ' + str(groupedIDs[index]) + ' ... and ...' + str(groupedIDs[group_index]))
                adjMat[group_index,index] = 1

[0, 6731, 13930, 16261, 18532, 21614]
[1, 21618]
[1, 21618]
[1, 21618]
[1, 21618]
[2]
[2]
[2]
[3, 3635, 6734, 10227, 13933, 16264, 18534, 21616]
[4, 3636, 6735, 10228, 13934, 16265, 18535, 21617]
[4, 3636, 6735, 10228, 13934, 16265, 18535, 21617]
[4, 3636, 6735, 10228, 16265, 18535, 21617]
[5, 3637, 6737, 10236, 13940, 16271, 18542, 21625]
[5, 6737, 10236, 13940, 16271, 18542, 21624]
[5, 6737, 10236, 13940, 16271, 18542, 21624]
[6, 4127, 10229, 18553, 21637]
[6, 4127, 10229, 18553, 21637]
[6, 4127, 10229, 18553, 21627]
[7, 3638, 6738, 10239, 13942, 16273, 18544, 21629]
[7, 3638, 6738, 10239, 13942, 16273, 18544, 21629]
[7, 3638, 6733, 10239, 13942, 16273, 18544, 21629]
[7, 3638, 6738, 10239, 13942, 16273, 18567, 21629]
[7, 3641, 6738, 10239, 13942, 16273, 18544, 21629]
[7, 3638, 6738, 13942, 16273, 21629]
[8, 6740, 10242, 13944, 16275, 18547, 21631]
[8, 6740, 10242, 13944, 16275, 18547, 21631]
[8, 3640, 6740, 10242, 13944, 16275, 18547, 21631]
[8, 3640, 6740, 10242, 13944, 16275, 18547

[172, 10442, 18701, 21792]
[173, 3785, 6927, 10443, 14057, 16383, 18702, 21793]
[174, 3786, 10446, 21796]
[175, 3787, 6928, 10447, 14059, 16385, 18704, 21797]
[176, 3788, 6929]
[177, 3789]
[178, 3790, 6930, 10448, 14060, 18705, 21798]
[179, 6931, 10450, 18707]
[180, 3791, 6932, 10451, 15595, 17886, 21800]
[180, 3791, 6932, 10451, 15595, 17886, 21800]
[181, 3792, 6933, 10451, 14061, 16386, 18708, 21800]
[182, 10452, 14062, 16387, 21801]
[183, 3793, 6934, 10453, 14063, 16388, 18709, 21802]
[184, 6935, 10455]
[185, 3794, 6936]
[186, 3795, 6865, 18711, 21804]
[186]
[187, 3796, 6937, 10456, 14065, 16390, 18712, 21806]
[188, 3797, 6938, 10457, 14066, 16391, 18713, 21807]
[189, 3798, 6939, 10458, 14067, 16392, 18714, 21808]
[190, 10459, 18715]
[191, 3799, 6940, 10460, 14068, 16393, 18716, 21809]
[192, 3800, 6941]
[193, 3801, 6942, 10461, 14069, 16394, 18717, 21810]
[194]
[195, 3802, 6943, 10463, 14070, 16395, 18719, 21812]
[196, 6944]
[196, 6944]
[196, 6944]
[196, 6944]
[197, 3803, 6945, 1046

[374]
[375, 3944, 7105, 10640, 14179, 16501, 18870, 21970]
[376, 7106, 10641, 14180, 18871, 21971]
[377, 3946, 7108, 10644, 14182, 16503, 18874]
[377, 3946, 7108, 10644, 18874]
[377, 3946, 7108, 10644, 18874]
[377, 3946, 7108, 14182, 16503]
[378, 3947, 7109, 10645]
[379, 3948, 7110]
[380, 7111]
[381, 7112, 18875, 23137]
[382, 3949, 7113, 10646, 14183, 16504, 18876, 21975]
[383, 18877, 21976]
[384]
[385, 3950, 10648, 21978]
[386, 3951, 7115]
[387, 3952, 7116, 10649, 14184, 16505, 18879, 21979]
[388, 3953, 7117, 10650, 21980]
[389, 3954, 7118, 10651, 14185, 16506, 18880, 21981]
[390, 3955, 7119, 10652, 14186, 16507, 18881, 21982]
[390, 3955, 7119, 10652, 14186, 16507, 21982]
[391, 7121, 10654, 14187, 16508, 18883, 21984]
[392, 3956]
[393, 3957, 7122, 10655, 18885]
[394, 3958, 7123]
[395, 3959]
[396, 3960, 7124, 10656, 18886, 21985]
[397, 3961, 7125, 10657, 14188, 16509, 18887, 21986]
[398, 3962, 7126, 10659, 14189, 16510, 18888, 21988]
[398, 3962, 7126, 10659, 14189, 16510, 18888]
[398, 

[576, 7304, 10852]
[576, 7304]
[576, 5197, 8510, 12123, 22154]
[576, 7304, 10852]
[576, 5197, 8510, 11175, 14563, 16877]
[576, 5197, 8510, 11175, 14563, 16877]
[576, 5197, 7304, 12123, 22154]
[576, 7304, 10852, 22154]
[576, 7304]
[576]
[576, 7304, 10852, 22154]
[576, 5197, 7304, 12123, 22154]
[576, 7304, 10852]
[576, 5197, 8510, 11175, 14563, 16877, 22154]
[576, 7304, 10852, 22154]
[576, 10852]
[576, 5197, 7304, 12123, 22154]
[576, 7304, 10852, 22154]
[576, 7304, 10852, 22154]
[576, 5197, 7304, 12123, 22154]
[576, 7304, 10852]
[576, 7304, 22154]
[576, 5197, 7304, 12123, 22154]
[576, 7304, 10852, 22154]
[576, 7304, 10852, 22154]
[576, 7304, 10852]
[576, 10852]
[576, 7304, 10852]
[576, 11028, 19210, 22336]
[576, 4209, 7304, 14389, 16709, 22154]
[576, 4209, 7304, 14389, 16709, 22154]
[576, 5197, 8510, 12123, 22154]
[576, 4209, 7410, 14389, 16709]
[577, 4120, 7409, 10838, 14301, 16620, 19137, 22259]
[577, 4120, 7305, 10838, 14301, 16620, 19038, 22155]
[577, 4120, 7409, 10838, 14301, 16620,

[674, 4203, 7404, 10938, 14386, 16706, 19132, 22255]
[674, 4203, 7404, 10938, 14386, 16706, 19132, 22255]
[675, 4204, 7405, 10939, 14387, 16707, 19133, 22256]
[675, 4204, 7405, 10939, 14387, 16707, 19133, 22256]
[675, 4204, 7405, 10939, 14387, 16707, 19133, 22256]
[675, 7405, 10939, 19133]
[675]
[676, 4205]
[676, 4205]
[677, 4207, 7407, 10942, 21541, 24904]
[677, 4207, 7407, 10942, 14837, 17150, 21541, 24904]
[677, 4207, 7407, 10942, 14837, 17150, 21541, 24904]
[678, 4208, 7408, 10943, 14388, 16708, 19136, 22258]
[679, 10944]
[680, 4209, 14389, 16709]
[680, 4209, 7410, 14389, 16709, 19138]
[681, 4210, 7411, 10945, 14390, 16710, 19139, 22260]
[682, 4211, 7412, 10946, 14391, 16711, 19140, 22261]
[683, 4212, 7413, 10947, 14392, 16712, 19141, 22262]
[684, 4213, 7414]
[684, 4213, 7414]
[685, 4214, 7415, 10949, 14394, 16714, 19143, 22264]
[686, 4215, 7416, 10951, 22265]
[687, 4216, 7417, 10953, 14395, 16715, 19144, 22266]
[688, 4217, 7418, 10954, 14396, 16716, 19145, 22267]
[689, 4219, 7419,

...

[1417, 8139, 14970, 17281, 19832, 22997]
[1418, 4874, 8140, 11727, 14971, 17282, 19833, 22998]
[1419, 4875, 8141, 11728, 19835, 22999]
[1421, 4876, 8143, 17283, 19836, 23000]
[1424, 4878, 8145, 11734, 14972, 17284, 19837, 23001]
[1425, 8146, 11735, 14973, 17285, 23002]
[11736, 23003]
[1427, 4880, 8148, 11737, 14974, 17286, 19838, 23004]
[961, 11738, 14975, 17287, 23005]
[1431, 4883, 8151, 11741, 14977, 17290, 19839, 23006]
[1432, 4884, 8152, 11742, 14978, 17291, 19840, 23007]
[19841, 23008]
[1433, 4885, 8153, 11743, 19842, 23009]
[1434, 4886, 8154, 11744, 14979, 17292, 19843, 23010]
[1436, 11745, 19844, 23011]
[1437, 4888, 8156, 23012]
[1439, 4890, 8158, 11746, 23013]
[1441, 4891, 8159, 11748, 14980, 17294, 19845, 23014]
[23015]
[1442, 4892, 8160, 11749, 14981, 17295, 19846, 23016]
[1443, 4893, 23017]
[1444, 8161, 11750, 19847, 23018]
[1445, 4894, 8162, 11751, 14982, 17296, 19848, 23019]
[1446, 4895, 8163, 11752, 14983, 17297, 19849, 23020]
[1447, 4896, 8164, 11753, 23021]
[8166, 11754

[1633, 5051, 8338, 11937, 15106, 17416, 20106, 23186]
[1634, 5052, 8339, 11938, 20006, 23187]
[1637, 5055, 8342, 11940, 15107, 17417, 20008, 23188]
[1638, 5056, 8343, 11941, 15108, 17418, 20009, 23189]
[1640, 5057, 8344, 11942, 15109, 17419, 20010, 23190]
[1641, 8345, 11943, 20011, 23191]
[3249, 8346, 11944, 15110, 17420, 20012, 23192]
[11945, 15111, 17421, 20013, 23193]
[1643, 5059, 11946, 23194]
[11947, 20014, 23195]
[5060, 8486, 12102, 20140, 23196]
[8486, 12102, 20140, 23196]
[5060, 12102, 23196]
[11948, 23197]
[11949, 23198]
[1644, 5061, 8349, 11950, 15112, 17422, 20015, 23199]
[1645, 5062, 8350, 11951, 15113, 17423, 20016, 23200]
[11952, 23201]
[1646, 5063, 8351, 11953, 20017, 23202]
[1647, 5064, 8352, 11954, 23203]
[1649, 5065, 8353, 11955, 15114, 17424, 20019, 23204]
[1651, 5067, 8355, 11956, 15116, 17426, 20021, 23205]
[1652, 5068, 8356, 11957, 15117, 17427, 20022, 23206]
[1653, 5069, 8357, 11958, 15118, 17428, 20023, 23207]
[1654, 8358, 11959, 15119, 17429, 20024, 23208]
[165

[12154, 20184, 23378]
[5233, 12155, 15241, 17547, 23379]
[1849, 5234, 8546, 12156, 15242, 17548, 20185, 23380]
[1850, 5235, 8547, 12157, 15243, 17549, 20186, 23381]
[1851, 5236, 8548, 12159, 15244, 17550, 20187, 23382]
[1851, 5236, 8548, 12159, 15244, 17550, 20187, 23382]
[1853, 5237, 8549, 12160, 15245, 17551, 20188, 23383]
[12161, 23384]
[12162, 23385]
[8550, 12163, 15246, 17552, 20189, 23386]
[1854, 5239, 8552, 12165, 15247, 17553, 20191, 23387]
[1855, 5241, 8553, 12166, 20192, 23388]
[1856, 5242, 8554, 12167, 20193, 23389]
[12168, 15248, 17554, 20194, 23390]
[12172, 20198, 23391]
[644, 4180, 7375, 12174, 15250, 17556, 20199, 23392]
[1862, 8560, 12175, 15251, 17557, 20200, 23393]
[12176, 23394]
[1863, 5248, 8561, 15252, 17558, 20201, 23395]
[1864, 5249, 8562, 12177, 15253, 17559, 20202, 23396]
[1865, 5250, 8563, 12178, 15254, 17560, 20203, 23397]
[1870, 5252, 8565, 12181, 15255, 17561, 20207, 23398]
[1871, 12182, 20208, 23399]
[1872, 5253, 12183, 20209, 23400]
[1873, 8566, 12184, 15

[2093, 5431, 8764, 12393, 15361, 17663, 20371, 23592]
[2094, 5432, 8765, 12394, 15362, 17664, 20372, 23593]
[2095, 12395, 20373, 23594]
[23595]
[2096, 5433, 8766, 12396, 15363, 17665, 20374, 23596]
[12397, 23597]
[1954, 5312, 8631, 12255, 15292, 17596, 20262, 23598]
[1954, 5312, 8631, 12255, 15292, 17596, 20262, 23598]
[2097, 5434, 8767, 12398, 15641, 17929, 20375, 23599]
[12401, 23600]
[8769, 12402, 20377, 23601]
[2100, 5437, 8770, 12403, 23602]
[2103, 5440, 8773, 12404, 15364, 17666, 20378, 23603]
[2105, 5443, 8775, 12405, 15365, 17667, 20379, 23604]
[2108, 8778, 12407, 15366, 17668, 20381, 23605]
[23606]
[2110, 5447, 8780, 12410, 20382, 23607]
[2111, 5448, 8781, 12411, 15368, 17670, 20383, 23608]
[2112, 5449, 8782, 12412, 15369, 17671, 20384, 23609]
[12414, 23610]
[2115, 5450, 8784, 12415, 15370, 17672, 20388, 23611]
[20389, 23612]
[23613]
[2117, 5451, 8785, 12416, 20390, 23614]
[2119, 8787, 12418, 15371, 17673, 20392, 23615]
[2121, 5454, 8789, 12419, 15372, 17674, 20393, 23616]
[21

KeyboardInterrupt: 

In [29]:
np.sum(adjMat) #Need to verify that this is correct...

130228.0

In [54]:
graph = networkx.from_numpy_matrix(adjMat)

In [59]:
subgraphs = [list(graph.subgraph(c).copy().nodes()) for c in networkx.connected_components(graph)]


In [60]:
#Final function for Part 1
def generate_linked_groups(groupedIDs):
    '''
        Takes a list of protein groups and generates linked protein groups based on shared proteins.
        Generates an adjacency matrix for protein groups where:
            0 = no shared proteins between groups
            1 = shared proteins between groups
        And then finds connected subgraphs for that network. Each subgraph is a set of linked groups.
        groupedIDs: list of strings - protein IDs column from protein groups file. Proteins within a group should be separated by ';'
        returns: list of lists of ints - each element is a list of indexes protein groups in a linked group
    '''
    adjMat = np.zeros((len(groupedIDs),len(groupedIDs))) #Initializing the adjacency matrix
    
    #Iterate over each protein group, and find other protein groups that contain the same proteins.
    for group_index in range(len(groupedIDs)):
        for protein in groupedIDs[group_index].split(';'):
            
            # also in is a list of indexes of protein groups that share a protein with this group.
            also_in  = list(np.where([protein in group for group in proteinGroups['Protein IDs']])[0])
            
            # Iterate through also_in and update the adjacency matrix
            for index in also_in:
                if group_index != index:
                    adjMat[group_index,index] = 1
    
    # Generate a list of indexes for component subgraphs
    graph = networkx.from_numpy_matrix(adjMat)
    return [list(graph.subgraph(c).copy().nodes()) for c in networkx.connected_components(graph)]
    

In [61]:
generate_linked_groups(list(proteinGroups['Protein IDs']))

[[0],
 [1],
 [2],
 [3],
 [4],
 [5],
 [6],
 [7],
 [8],
 [9],
 [10],
 [11],
 [12],
 [13],
 [14],
 [15],
 [16],
 [17],
 [18],
 [19],
 [20],
 [21],
 [22],
 [23],
 [24],
 [25],
 [26],
 [27],
 [28],
 [29],
 [30],
 [31],
 [32],
 [33],
 [34],
 [35],
 [36],
 [37],
 [38],
 [39],
 [40],
 [41],
 [42],
 [43],
 [44],
 [45],
 [46],
 [47],
 [48],
 [49],
 [50],
 [51],
 [52],
 [53],
 [54],
 [55],
 [56],
 [57],
 [58],
 [59],
 [60],
 [61],
 [62],
 [63],
 [64],
 [65],
 [66],
 [67],
 [68],
 [69],
 [70],
 [71],
 [72],
 [73],
 [74],
 [75],
 [76],
 [77],
 [78],
 [79],
 [80],
 [81],
 [82],
 [83],
 [84],
 [85],
 [337, 86],
 [421, 87],
 [88],
 [89, 527],
 [90],
 [91],
 [92],
 [760, 93, 327],
 [813, 94],
 [95],
 [96],
 [904, 97],
 [98],
 [99],
 [100],
 [101],
 [102],
 [103],
 [104, 1268],
 [105, 1284, 4486],
 [106, 1358],
 [107, 1375],
 [108],
 [109, 1566],
 [110],
 [111],
 [112, 1808],
 [113],
 [114],
 [115],
 [116],
 [117],
 [118, 2070],
 [2089, 119],
 [120, 1009],
 [2272, 121],
 [122],
 [123, 2303],
 [124],
 [1

In [23]:
testword = 'spice'
test = ['spice','meat','toast']

In [24]:
testword in test

True

In [14]:
also_in

[4798]